In [34]:
# https://www.youtube.com/watch?v=xdq6Gz33khQ&ab_channel=CodingEntrepreneurs
# Coneccion a la API de Spotify
client_id = '2fcf86918d5d434c90fe82b1294d12b0'
client_secret = 'd7fa0a4a43fb4fff9cc3bce1940ff876'
user_id = '1279741536'

In [35]:
import base64
import requests
import datetime
from urllib.parse import urlencode

class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    client_id = None
    client_secret = None
    access_token_did_expire = True
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args,**kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    # Aqui se crean las credenciales en base 64 para poder acceder a la API
    def get_client_credentials(self):
        """ returns a base64 encoded string """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
            
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())        
        return client_creds_b64.decode()
    
    # Aqui estamos generando los headers de acceso
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        
        return {
            "Authorization":f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type":"client_credentials"
        }
    
    # Aqui nos autentificamos y obtenemos el token de acceso
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        
        r = requests.post(token_url, data = token_data, headers = token_headers)

        if r.status_code not in range(200,299):
            raise Exceptiontion('Could not authenticate client')
                  
        data = r.json()
        now = datetime.datetime.now()
        acces_token = data['access_token']
        expires_in = data['expires_in'] #seconds
        expires = now + datetime.timedelta(seconds = expires_in)
        self.access_token = acces_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    # Extraemos el token de acceso
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    # Aqui son los tokens de para las busquedas
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization":f"Bearer {access_token}"
        }
        return headers
        
    # busqueda de listas de reproduccion
    def get_user_playlist(self,u_id):
        endpoint = f'https://api.spotify.com/v1/users/{user_id}/playlists'
        headers = self.get_resource_header()
        r = requests.get(endpoint,headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def get_song_playlist(self,playlist_id):
        endpoint = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
        headers = self.get_resource_header()
        r = requests.get(endpoint,headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    # de Aqui en adelante son las busquedas que se requieren
    def get_resource(self,lookup_id,resource_type ='albums',version='v1'):
        endpoint = f'https://api.spotify.com/{version}/{resource_type}/{lookup_id}'
        headers = self.get_resource_header()
        r = requests.get(endpoint,headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type = 'albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type = 'artists')
    
    def base_search(self,query_params):
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers = headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, search_type='artist' ):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower()})
        return self.base_search(query_params)

In [36]:
spotify = SpotifyAPI(client_id,client_secret)

In [37]:
spotify.get_user_playlist(user_id)

{'href': 'https://api.spotify.com/v1/users/1279741536/playlists?offset=0&limit=20',
 'items': [{'collaborative': False,
   'description': '',
   'external_urls': {'spotify': 'https://open.spotify.com/playlist/4cneXKJlVKFLKAYuwtk57I'},
   'href': 'https://api.spotify.com/v1/playlists/4cneXKJlVKFLKAYuwtk57I',
   'id': '4cneXKJlVKFLKAYuwtk57I',
   'images': [{'height': 640,
     'url': 'https://mosaic.scdn.co/640/ab67616d0000b273621489fd90d50158b3714cb0ab67616d0000b273627068a6288cf15faaa7e296ab67616d0000b273c4d00cac55ae1b4598c9bc90ab67616d0000b273d5929527336d3a19add3e036',
     'width': 640},
    {'height': 300,
     'url': 'https://mosaic.scdn.co/300/ab67616d0000b273621489fd90d50158b3714cb0ab67616d0000b273627068a6288cf15faaa7e296ab67616d0000b273c4d00cac55ae1b4598c9bc90ab67616d0000b273d5929527336d3a19add3e036',
     'width': 300},
    {'height': 60,
     'url': 'https://mosaic.scdn.co/60/ab67616d0000b273621489fd90d50158b3714cb0ab67616d0000b273627068a6288cf15faaa7e296ab67616d0000b273c4d00c

{'error': {'status': 401, 'message': 'No token provided'}}